## Preface
A blackbox function is a function who's exact behavior is unknown. One can only pass the function input and it will return an output. Despite not knowing the function's exact behavior, and thus not knowing any of its derivatives, there are still techniques that may allow one to locate the function's global extrema. 

### Technique Overview
To *briefly* summarize the blackbox function optimization techniques we will be examining:
#### Grid Search
- "Brute Force" method
- Iteratively evaluate blackbox function at all combinations of input values to find the input that yields the highest function evaluation
#### Random Sampling
- By taking random inputs and comparing their function evaluations, this techniques offers a way to locate global extremma without iterating over the entire hyperparameter space. This means a significant reduction in fine-tunning time from Grid Search, which can be a neccessity when working with blackbox functions that have larger domains and/or blackbox functions that take a long time to evaluate. For this neural network application, both are the case. 
#### Quasi-random Sampling
- Similar to Random Sampling, but quasi-random sequences are used to avoid areas of overly sparse sampling and overly dense sampling in the blackbox function's domain. 
#### Bayesian Optimization
- A technique that makes a prior probability distribution of where it believes the blackbox function is. As it recieves more evaluations of the blackbox function, Bayesian Optimization produces better probability distributions about where the function can be. The inputs evaluated in the (n-1)-th step are chosen with an aquisition function given the output probability distribution over f found in the n-th step. Different aquisition functions locates a candidate for the next evaluation by maximizing different criteria.

## Objective
Assuming the the relation between a neural network's hyperparameters and it's accuracy on test_data is such a blackbox function, *f*, this notebook explores the effects of using various blackbox function optimization techniques on neural network hyperparameters including:
- Random Sampling
- Quasi-random Sampling
- Bayesian Optimization

The calculations required to run a Grid Search will provide motivation for the above techniques.

The hyperparameters (input to *f*) explored here will include:
- The number of hidden layers
- The number of nodes in each hidden layer
- The number of epochs neural network is trained over

The data that will be used is Tensorflow's Fashion_mnist dataset.
The neural network structure is based on [this](https://www.tensorflow.org/tutorials/keras/classification) basic Tensorflow tutorial:
- A 784-node Flatten Layer
- Hidden Dense Layer(s) with varying numbers of nodes
- A 10-node Dense Layer

The neural network in the tutorial contained only one hidden layer with 128 nodes. It was trained over 10 epochs.

### Hypothesis
It is expected that Bayesian Optimization will perform the best, the Quasi-Random Sampling Will perform the second best, and Random-Sampling will perform the third best. 



## NonObjective Components
Imports:

In [3]:
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import sobol
import skopt
%matplotlib inline

2021-07-04 09:21:30.577677: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-04 09:21:30.577720: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Import Fashion mnist dataset. Image data is black and white, and the pixel intensities are scaled to be between 0 and 1.

In [4]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = train_images / 255.0
test_images = test_images / 255.0

`model()` is a function to generate a neural network model based on neural network structure hyperparameters

In [5]:
def model(nodes_in_layer_1, nodes_in_layer_2, nodes_in_layer_3, nodes_in_layer_4, nodes_in_layer_5, input_shape=(28, 28), activation='relu',
          optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
          metrics=('accuracy')):
      nodes_per_layer = (nodes_in_layer_1, nodes_in_layer_2, nodes_in_layer_3, nodes_in_layer_4, nodes_in_layer_5)
      layers = [tf.keras.layers.Flatten(input_shape=input_shape)]
      for num_of_nodes in nodes_per_layer:
            if num_of_nodes != 0:
                  layers.append(tf.keras.layers.Dense(num_of_nodes, activation=activation))
      layers += [tf.keras.layers.Dense(10)]
      model = tf.keras.Sequential(layers)
      model.compile(optimizer=optimizer,
                    loss=loss,
                    metrics=metrics)
      return model

`evaluate_model` accepts neural network hyperparameters and outputs accuracy of model's accuracy on test_data. This method represents *f*.

In [6]:
def evaluate_model(nodes_in_layer_1, nodes_in_layer_2, nodes_in_layer_3, nodes_in_layer_4, nodes_in_layer_5, epochs,
                   train_x=train_images, train_y=train_labels, test_x=test_images, test_y=test_labels):
      current_model = model(nodes_in_layer_1, nodes_in_layer_2, nodes_in_layer_3, nodes_in_layer_4, nodes_in_layer_5)
      current_model.fit(train_x, train_y, epochs=epochs, verbose=0)
      loss, accuracy = current_model.evaluate(test_x, test_y, verbose=0)
      return accuracy

Domain of *f* will be subject to the following restrictions:
- nodes_per_layer in [20, 150]
- layers in [1, 5]
- epochs [10, 50]

The subset of the domain subject to these restrictions will be referenced as the domain of *f* for simplicity.
This means the domain contains (150 - 20 + 1)(5 - 1 + 1)(50 - 10 + 1) = (131)(5)(41) = 26855 elements.

In [7]:
NODES_PER_LAYER_DOMAIN = (20, 150)
NUM_OF_LAYERS_DOMAIN = (1, 5)
NUM_OF_EPOCHS_DOMAIN = (10, 50)

## Control
As stated in the objective, the neural network in the base tutorial will be used as a control. To reiterate, it comprised of only one hidden layer with 128 nodes and was trained over 10 epochs. 

In [9]:
tutorial_model_eval = evaluate_model(128, 0, 0, 0, 0, 10)
tutorial_model_eval

0.8723999857902527

As seen above, a neural network with the base tutorial hyperparameters performed with an 87.240% test accuracy. Although it is not calculation intensive to evaluate the tutorial network multiple times use the mean test accuracy, this exploration will not do so in order to maintain consistency with the evaluation of the other techniques. The other techniques are run only once due to limited computation resources.

## Grid Search
Only the hyperparameters combinations in the domain will be generated for Grid Search because (as will become evident shortly) Grid Search is unreasonably calculation intensive.

In [ ]:
def perm(n, domain):
    permutations = []
    for x in domain:
        for i in range(len(domain) ** (n - 1)):
            permutations.append([x,])
    for col in range(1, n):
        column = []
        for chunk in range(len(domain) ** (col + 1)):
            column += ([domain[chunk % len(domain)] for i in range(len(domain) ** (n - col - 1))])
        for i in range(len(column)):
            permutations[i].append(column[i])
    return permutations

model_hyperparams = []
for num_of_epochs in range(NUM_OF_EPOCHS_DOMAIN[0], NUM_OF_EPOCHS_DOMAIN[1] + 1):
    for layers in range(NUM_OF_LAYERS_DOMAIN[0], NUM_OF_LAYERS_DOMAIN[1] + 1):
        current_epoch_hyperparams = perm(layers, [i for i in range(NODES_PER_LAYER_DOMAIN[0], NODES_PER_LAYER_DOMAIN[1] + 1)])
        for structure_index in range(len(current_epoch_hyperparams)):
            current_epoch_hyperparams[structure_index] = [current_epoch_hyperparams[structure_index], num_of_epochs]
        model_hyperparams += current_epoch_hyperparams
len(model_hyperparams)

As seen above, for a reasonable domain of *f* the number of times the calculation intensive *f* must be evaluated is extremely large. In this case, the kernal crashes before all the hyperparameters can be created. The subset of the domain contains 26855 elements and would require that *f* be evaluated ((150 - 20 + 1)^3 + (150 - 20 + 1)^2 + (150 - 20 + 1)^1)(5) = 11326915 times.

## Random Sampling
By selecting a random subset of the domain of *f*, we significantly reduce the calculation intensity of blackbox optimization. Here, the random sample size was chosen to be .01% of the total domain of *f*. (11326915 * 0.0001) = 1132.69 which rounds to 1133 samples.

In [33]:
RANDOM_SAMPLE_SIZE = 5

random_samples = []
for sample_index in range(RANDOM_SAMPLE_SIZE):
    layers = []
    total_layers = int(random.randint(NUM_OF_LAYERS_DOMAIN[0], NUM_OF_LAYERS_DOMAIN[1]))
    for layer_index in range(total_layers):
        layers.append(random.randint(NODES_PER_LAYER_DOMAIN[0], NODES_PER_LAYER_DOMAIN[1]))
    if len(layers) != 5:
        layers += [0] * (5 - len(layers))
    random_samples.append({'layers': layers, 'epochs': random.randint(NUM_OF_EPOCHS_DOMAIN[0], NUM_OF_EPOCHS_DOMAIN[1])})
        

In [48]:
def evaluate_samples(samples, train_x, train_y, test_x, test_y):
      most_accurate_sample_model = {'accuracy': 0}
      random_sample_eval = []
      for sample_index in range(len(samples)):
            print(f"Training sample {sample_index + 1}/{RANDOM_SAMPLE_SIZE}")
            sample_model_accuracy = evaluate_model(*samples[sample_index]['layers'], samples[sample_index]['epochs'], train_x, train_y, test_x, test_y)
            random_sample_eval.append({'accuracy': sample_model_accuracy, 'model': samples[sample_index]})
            if sample_model_accuracy > most_accurate_sample_model['accuracy']:
                  most_accurate_sample_model['accuracy'] = sample_model_accuracy
                  most_accurate_sample_model['hyperparameters'] = samples[sample_index]
      return {'sample_eval': random_sample_eval, 'most_accurate_sample_model': most_accurate_sample_model}
print(f'Most accurate model from Random Sampling: {evaluate_samples(random_samples, train_images, train_labels, test_images, test_labels)}')

Training sample 1/5
triggered
Training sample 2/5
Training sample 3/5
triggered
Training sample 4/5
Training sample 5/5
Most accurate model from Random Sampling: {'sample_eval': [{'accuracy': 0.8851000070571899, 'model': {'layers': [39, 102, 47, 0, 0], 'epochs': 14}}, {'accuracy': 0.875, 'model': {'layers': [37, 0, 0, 0, 0], 'epochs': 22}}, {'accuracy': 0.8891000151634216, 'model': {'layers': [147, 0, 0, 0, 0], 'epochs': 35}}, {'accuracy': 0.8806999921798706, 'model': {'layers': [139, 0, 0, 0, 0], 'epochs': 11}}, {'accuracy': 0.8859000205993652, 'model': {'layers': [78, 30, 54, 31, 0], 'epochs': 24}}], 'most_accurate_sample_model': {'accuracy': 0.8891000151634216, 'hyperparameters': {'layers': [147, 0, 0, 0, 0], 'epochs': 35}}}


## Quasi-random Sampling
The Sobol Sequence is the Quasi-random sequence used to ensure that the random samples do not leave overly sparse or overly dense areas of sampling.

In [ ]:
sobol_samples = sobol.sample(dimension=6, n_points=RANDOM_SAMPLE_SIZE)
for sobol_point in sobol_samples:
    for sobol_point_index in range(len(sobol_point) - 1):
        sobol_point[sobol_point_index] = sobol_point[sobol_point_index] * (NODES_PER_LAYER_DOMAIN[1] - NODES_PER_LAYER_DOMAIN[0]) + NODES_PER_LAYER_DOMAIN[0]
        sobol_point[-1] = sobol_point[sobol_point_index] * (NUM_OF_EPOCHS_DOMAIN[1] - NUM_OF_EPOCHS_DOMAIN[0]) + NUM_OF_EPOCHS_DOMAIN[0]

print(f'Most accurate model from Random Sampling: {evaluate_samples(sobol_samples, train_images, train_labels, test_images, test_labels)}')

## Bayesian Optimization
Scikit's gp_minimize is used to find global minimum via Bayesian Optimization. The global minimum of *-f* is the global maximimum of *f*

In [57]:
def f(x):
    return -evaluate_model(*x)
domains = [(NODES_PER_LAYER_DOMAIN[0], NODES_PER_LAYER_DOMAIN[1]),] * 5 + [(NUM_OF_EPOCHS_DOMAIN[0], NUM_OF_EPOCHS_DOMAIN[1]),]
global_maximium = skopt.gp_minimize(f, domains, acq_func='EI', n_calls=20, n_random_starts=5, random_state=1234)
print(global_maximium.x, global_maximium.fun)

[(20, 150), (20, 150), (20, 150), (20, 150), (20, 150), (10, 50)]
[150, 107, 102, 145, 76, 50] -0.8928999900817871


# Conclusion
In conclusion, using blackbox optimization on neural network hyperparameters did result in a large improvement over the tutorial hyperparameters. However, it is important to note that these conclusions only apply to the structural hyperparameters examined here and the number of epochs over which the network is trained. Since the control was the tutorial network, it can be concluded that the tutorial network's number of layers, nodes per layer, and epochs trained provided a close to optimal solution. However, these techniques could provide *a way* to get a network with decent performance if there are no other starting points to the network's hyperparameters.

Although Grid Search could provide a guarunteed best performance, it is so calculation intensive that it becomes unfeasable. The other three techniques examined provide more calculation efficient, and thus reasonable, solutions.

Future explorations can improve upon this one by examining each technique over multiple trials allowing the compaison of means, standard deviations, graphical visualizations, and other interpretations of the data. This would provide more insight into the nature of these techniques and also provide more accurate evaluations of the techniques, especially the techniques involving randomness. Improvements can also be made in the depth at which the individual techniques are examined. Bayesian optimization can run with various ways of evaluating which point is best to evaluate next via different acquisition functions. Quasi-Random Search can use different techniques to distribute random samples. Finally, there are other blackbox optimization techniques that could be examined such as Parzen Tree Eestimators.

## References
https://en.wikipedia.org/wiki/Sobol_sequence

https://en.wikipedia.org/wiki/Low-discrepancy_sequence

https://machinelearningmastery.com/how-to-improve-neural-network-stability-and-modeling-performance-with-data-scaling/

http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html

https://www.borealisai.com/en/blog/tutorial-8-bayesian-optimization/

https://medium.com/@hiepnguyen034/improving-neural-networks-performance-with-bayesian-optimization-efbaa801ad26

Walz, David(2020) SobolSequence Python Module (Version 0.2) https://pypi.org/project/SobolSequence/

